## UNData API Exercise

In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).

1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [2]:
import requests
import pandas as pd

def get_worldbank_data(indicator_code):
    base_url = "https://api.worldbank.org/v2/country/all/indicator/"
    format = "json"
    per_page = 20000  
    
    url = f"{base_url}{indicator_code}?format={format}&per_page={per_page}"
    
    response = requests.get(url)
    data = response.json()
    
    records = data[1]
    
    df = pd.DataFrame(records)
    
    df['country_name'] = df['country'].apply(lambda x: x['value'])
    
    df = df[['country_name', 'countryiso3code', 'date', 'value']]
    df = df.rename(columns={
        'country_name': 'Country',
        'countryiso3code': 'CountryCode',
        'date': 'Year',
        'value': 'GDP_per_capita_PPP'
    })
    
    return df

gdp_df = get_worldbank_data("NY.GDP.PCAP.PP.KD")

print(gdp_df.head())

                       Country CountryCode  Year  GDP_per_capita_PPP
0  Africa Eastern and Southern         AFE  2023         3967.860937
1  Africa Eastern and Southern         AFE  2022         3974.803045
2  Africa Eastern and Southern         AFE  2021         3934.287267
3  Africa Eastern and Southern         AFE  2020         3861.111238
4  Africa Eastern and Southern         AFE  2019         4073.880522


2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [4]:
life_expectancy_data = get_worldbank_data("SP.DYN.LE00.IN")

life_df = life_expectancy_data.rename(columns={'GDP_per_capita_PPP': 'LifeExpectancy'})
life_df = life_df.dropna(subset=['LifeExpectancy']) 

print(life_df.head())

                       Country CountryCode  Year  LifeExpectancy
1  Africa Eastern and Southern         AFE  2022       62.888463
2  Africa Eastern and Southern         AFE  2021       62.449093
3  Africa Eastern and Southern         AFE  2020       63.309794
4  Africa Eastern and Southern         AFE  2019       63.754752
5  Africa Eastern and Southern         AFE  2018       63.367037


4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [6]:
def get_country_data():
    url = "https://api.worldbank.org/v2/country?format=json&per_page=300"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()[1]
    except Exception as e:
        print(f"Error fetching country data: {e}")
        return pd.DataFrame()

    country_data = []
    for country in data:
        region = country.get('region', {})
        entry = {
            'CountryCode': country.get('iso3Code', country.get('id')), 
            'CountryCode2': country.get('iso2Code'),
            'Country': country.get('name'),
            'Region': region.get('value'),
            'RegionCode': region.get('id'),
            'IncomeLevel': country.get('incomeLevel', {}).get('value'),
            'LendingType': country.get('lendingType', {}).get('value'),
            'CapitalCity': country.get('capitalCity'),
            'Longitude': country.get('longitude'),
            'Latitude': country.get('latitude'),
            'IsCountry': region.get('id') not in ['NA', ''],
        }
        
        if entry['CountryCode'] and entry['Country']:
            country_data.append(entry)
    
    country_df = pd.DataFrame(country_data)
    return country_df[country_df['IsCountry']].copy()  # Return only countries

# Usage
country_df = get_country_data()
country_df.drop(columns=['IsCountry'], inplace=True, errors='ignore')
country_df

,CountryCode,CountryCode2,Country,Region,RegionCode,IncomeLevel,LendingType,CapitalCity,Longitude,Latitude
0,ABW,AW,Aruba,Latin America & Caribbean,LCN,High income,Not classified,Oranjestad,-70.0167,12.5167
2,AFG,AF,Afghanistan,South Asia,SAS,Low income,IDA,Kabul,69.1761,34.5228
5,AGO,AO,Angola,Sub-Saharan Africa,SSF,Lower middle income,IBRD,Luanda,13.242,-8.81155
6,ALB,AL,Albania,Europe & Central Asia,ECS,Upper middle income,IBRD,Tirane,19.8172,41.3317
7,AND,AD,Andorra,Europe & Central Asia,ECS,High income,Not classified,Andorra la Vella,1.5218,42.5075
...,...,...,...,...,...,...,...,...,...,...
290,XKX,XK,Kosovo,Europe & Central Asia,ECS,Upper middle income,IDA,Pristina,20.926,42.565
292,YEM,YE,"Yemen, Rep.",Middle East & North Africa,MEA,Low income,IDA,Sana'a,44.2075,15.352
293,ZAF,ZA,South Africa,Sub-Saharan Africa,SSF,Upper middle income,IBRD,Pretoria,28.1871,-25.746
294,ZMB,ZM,Zambia,Sub-Saharan Africa,SSF,Lower middle income,IDA,Lusaka,28.2937,-15.3982


In [7]:
UN_df = pd.merge(
    gdp_df,
    life_df,
    on=['CountryCode', 'Year', 'Country'],  
    how='inner',
    validate='one_to_one')

UN_df = pd.merge(
    UN_df,
    country_df,
    on=['CountryCode', 'Country'],  
    how='right')

UN_df.drop(columns=['CountryCode2', 'Country_y'], errors='ignore', inplace=True)

UN_df.head()

,Country,CountryCode,Year,GDP_per_capita_PPP,LifeExpectancy,Region,RegionCode,IncomeLevel,LendingType,CapitalCity,Longitude,Latitude
0,Aruba,ABW,2022,38877.674436,74.992,Latin America & Caribbean,LCN,High income,Not classified,Oranjestad,-70.0167,12.5167
1,Aruba,ABW,2021,35696.309372,74.626,Latin America & Caribbean,LCN,High income,Not classified,Oranjestad,-70.0167,12.5167
2,Aruba,ABW,2020,28521.688852,75.723,Latin America & Caribbean,LCN,High income,Not classified,Oranjestad,-70.0167,12.5167
3,Aruba,ABW,2019,38435.426913,76.248,Latin America & Caribbean,LCN,High income,Not classified,Oranjestad,-70.0167,12.5167
4,Aruba,ABW,2018,39419.555348,76.072,Latin America & Caribbean,LCN,High income,Not classified,Oranjestad,-70.0167,12.5167


**Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

1. Adjust your request so that it returns data just for the United States.

In [9]:
usa_df = UN_df[UN_df['CountryCode'] == 'USA'].copy()
usa_df

,Country,CountryCode,Year,GDP_per_capita_PPP,LifeExpectancy,Region,RegionCode,IncomeLevel,LendingType,CapitalCity,Longitude,Latitude
12223,United States,USA,2022,72841.924313,77.434146,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12224,United States,USA,2021,71318.307359,76.329268,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12225,United States,USA,2020,67352.387139,76.980488,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12226,United States,USA,2019,69511.763441,78.787805,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12227,United States,USA,2018,68070.213422,78.639024,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
...,...,...,...,...,...,...,...,...,...,...,...,...
12281,United States,USA,1964,NaN,70.165854,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12282,United States,USA,1963,NaN,69.917073,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12283,United States,USA,1962,NaN,70.119512,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895
12284,United States,USA,1961,NaN,70.270732,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895


2. Adjust your request so that it returns data just for the United States for the year 2021.

In [40]:
UN_df['Year'] = pd.to_numeric(UN_df['Year'], errors='coerce')

usa_2021 = UN_df[
    (UN_df['CountryCode'] == 'USA') & 
    (UN_df['Year'] >= 2000) & 
    (UN_df['Year'] <= 2021)
].copy()
usa_2021

,Country,CountryCode,Year,GDP_per_capita_PPP,LifeExpectancy,Region,RegionCode,IncomeLevel,LendingType,CapitalCity,Longitude,Latitude
12224,United States,USA,2021,71318.307359,76.329268,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895


3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

In [48]:
UN_df['Year'] = pd.to_numeric(UN_df['Year'], errors='coerce').astype('Int64')

usa_2021 = UN_df[
    (UN_df['CountryCode'] == 'USA') & 
    (UN_df['Year'] == 2021)
].copy()
usa_2021

,Country,CountryCode,Year,GDP_per_capita_PPP,LifeExpectancy,Region,RegionCode,IncomeLevel,LendingType,CapitalCity,Longitude,Latitude
12224,United States,USA,2021,71318.307359,76.329268,North America,NAC,High income,Not classified,Washington D.C.,-77.032,38.8895


4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [56]:
usa_can = UN_df[
    (UN_df['CountryCode'].isin(['USA', 'CAN'])) & 
    (UN_df['Year'] >= 2000) & 
    (UN_df['Year'] <= 2021)].copy()
usa_can

,Country,CountryCode,Year,GDP_per_capita_PPP,LifeExpectancy,Region,RegionCode,IncomeLevel,LendingType,CapitalCity,Longitude,Latitude
1893,Canada,CAN,2021,56547.800561,81.587073,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1894,Canada,CAN,2020,54006.584430,81.660488,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1895,Canada,CAN,2019,57491.982046,82.223902,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1896,Canada,CAN,2018,57246.020181,81.843902,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1897,Canada,CAN,2017,56522.014672,81.832927,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1898,Canada,CAN,2016,55517.446212,81.924878,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1899,Canada,CAN,2015,55572.071531,81.815610,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1900,Canada,CAN,2014,55634.587657,81.784390,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1901,Canada,CAN,2013,54624.895883,81.744878,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215
1902,Canada,CAN,2012,53948.601539,81.663659,North America,NAC,High income,Not classified,Ottawa,-75.6919,45.4215


5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

In [62]:
def get_all_worldbank_data(indicator_code, value_column_name):
    base_url = "https://api.worldbank.org/v2/country/all/indicator/"
    format = "json"
    per_page = 20000 
    page = 1
    all_records = []
    
    while True:
        url = f"{base_url}{indicator_code}?format={format}&per_page={per_page}&page={page}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            # Check if page has data
            if len(data) < 2:
                break
                
            # Process records with your country code logic
            for record in data[1]:
                processed = {
                    'CountryCode': record.get('countryiso3code', record.get('country', {}).get('id')),
                    'Country': record.get('country', {}).get('value'),
                    'Year': record.get('date'),
                    value_column_name: record.get('value')
                }
                if processed['CountryCode'] and processed['Country']:
                    all_records.append(processed)
            
            # Exit loop if last page
            if len(data[1]) < per_page:
                break
                
            page += 1
            
        except Exception as e:
            print(f"Error on page {page}: {e}")
            break
    
    # Create DataFrame
    df = pd.DataFrame(all_records)
    
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['CountryCode'] = df['CountryCode'].str.upper().str.strip()
    
    return df.dropna(subset=['CountryCode', 'Year'])

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.

In [98]:
def get_all_indicators():
    base_url = "http://api.worldbank.org/v2/indicator"
    all_indicators = []
    page = 1
    
    while True:
        url = f"{base_url}?format=json&page={page}&per_page=1000"
        response = requests.get(url)
        data = response.json()
        
        if not data[1]: 
            break
            
        all_indicators.extend(data[1])
        page += 1
    
    return all_indicators

all_indicators = get_all_indicators()

In [112]:
search_term = "Expenditure on Education"
data = requests.get("http://api.worldbank.org/v2/indicator?format=json&per_page=20000").json()
matches = [ind for ind in data[1] if search_term.lower() in str(ind).lower()] 

for m in matches:
    print(f"{m['id']}: {m['name']}")

PER.ALL.CPL.GDP: World Bank: Public capital expenditure on education as % of GDP
PER.ALL.CPL.TOT: World Bank: Public capital expenditure on education as a % of total capital government expenditure
PER.ALL.RCT.GDP: World Bank: Public recurrent expenditure on education (% of GDP)
PER.SOR.GOV.TOT.CPL: World Bank: Share of total capital expenditure on education (% from public sources)
PER.SOR.PVT.GDP: World Bank: Total private expenditure on education (% of GDP)
